In [27]:
import tensorflow as tf
import sys
print(sys.version)
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [28]:
class MF_RS():
    def __init__(self, numUsers, numSongs, embedding_dim, reg_lambda=0.01, conf_lambda=1.0):
        
        #hyper parameters
        self.batch_size = np.min([200, numUsers, numSongs]);
        self.numUsers = numUsers
        self.numSongs = numSongs
        self.epochs = 50
        self.reg_lambda = reg_lambda
        self.conf_lambda = conf_lambda
        
        #embedding matricies for users and songs
        self.userMat = tf.Variable(tf.random_normal([numUsers, embedding_dim]))
        self.songMat = tf.Variable(tf.random_normal([numSongs, embedding_dim]))
        self.userBias = tf.Variable(tf.random_normal([numUsers]))
        self.songBias = tf.Variable(tf.random_normal([numSongs]))
        self.overallBias = tf.Variable(tf.random_normal([1]))
        self.C_user = tf.Variable(.1*tf.ones([numUsers]))
        self.C_song = tf.Variable(.1*tf.ones([numSongs]))
        
        #input tensors for songs, usres, ratings
        self.users = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.songs = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.rating = tf.placeholder(tf.float32, shape = (self.batch_size))
        
        #map each user/song to its feature vector
        self.U = tf.nn.embedding_lookup(self.userMat, self.users)
        self.W = tf.nn.embedding_lookup(self.songMat, self.songs)
        # bias
        self.U_bias = tf.nn.embedding_lookup(self.userBias, self.users)
        self.W_bias = tf.nn.embedding_lookup(self.songBias, self.songs)
        # confidence params
        self.C_ui = tf.maximum(0.0, tf.nn.embedding_lookup(self.C_user, self.users))
        self.C_sj = tf.maximum(0.0, tf.nn.embedding_lookup(self.C_song, self.songs))

        
        #predicted rating is dot product of user and song
        bias = self.U_bias+self.W_bias+self.overallBias
        pq = tf.reduce_sum(tf.mul(self.U, self.W), 1)
        self.yhat = pq + bias
            
        # l2 reg
        self.confidence_reg = self.conf_lambda * tf.reduce_sum(tf.exp(-self.C_ui) + tf.exp(-self.C_sj))
        self.l2_reg = self.reg_lambda * ( tf.reduce_sum((tf.square(self.U) + tf.square(self.W))) + 
                                         tf.reduce_sum(tf.square(self.U_bias) + tf.square(self.W_bias)))
        self.reg = self.confidence_reg + self.l2_reg
        self.error = tf.reduce_mean(self.C_ui * self.C_sj * tf.nn.l2_loss(self.yhat - self.rating))
        self.cost = (self.error + self.reg)/1e7
        self.optimizer = tf.train.AdamOptimizer(learning_rate = .01).minimize(self.cost)
        
        self.session = tf.Session()
        self.session.run(tf.initialize_all_variables())    
        
    def train(self, users, songs, ratings):
        
        for i in range(self.epochs):
            
            avg_cost = 0
            perm = np.random.permutation(len(ratings))
            num_batches = len(ratings) // self.batch_size
            
            for b_idx in range(num_batches):
                
                batch = perm[self.batch_size * b_idx:self.batch_size * (b_idx + 1)]
                users_batch = users[batch]
                songs_batch = songs[batch]
                ratings_batch = ratings[batch]
                                
                avg_cost += self.session.run([self.cost, self.optimizer],
                                  {self.users:users_batch, self.songs:songs_batch, self.rating:ratings_batch})[0]
                
            print(avg_cost/num_batches)
    def test(self, users, songs):
        yhat = np.zeros(len(users))
        num_batches = len(users) // self.batch_size
        for b_idx in range(num_batches):
            batch = range(self.batch_size * b_idx,self.batch_size * (b_idx + 1))
            users_batch = users[batch]
            songs_batch = songs[batch]
            yhat[batch] = self.session.run([self.yhat],
                      {self.users:users_batch, self.songs:songs_batch})[0]
        batch = range(-self.batch_size,0)
        users_batch = users[batch]
        songs_batch = songs[batch]
        yhat[batch] = self.session.run([self.yhat],
                      {self.users:users_batch, self.songs:songs_batch})[0]
        return yhat
    def evaluate(self, users, songs, ratings):
        yhat = self.test(users, songs)
        return np.mean((yhat - ratings)**2)

In [29]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([1, 2, 3, 4, 5])
c = np.array([4, 3, 2, 5, 1])
#unique users / songs
uni_a = np.unique(a)
uni_b = np.unique(b)

#dict mapping the id to an index
a_map = dict(zip(uni_a,range(len(uni_a))))
b_map = dict(zip(uni_b,range(len(uni_b))))

user_idx =  np.array([ a_map[user] for user in a])
song_idx =  np.array([ b_map[song] for song in b])
model = MF_RS(len(uni_a), len(uni_b), 7)
np.random.seed(1)
model.train(user_idx, song_idx, c)


1.01680780062e-06
1.01670809727e-06
1.01656951301e-06
1.01640262073e-06
1.0162128774e-06
1.01600369362e-06
1.0157775705e-06
1.01553655441e-06
1.0152817822e-06
1.01501450445e-06
1.01473540326e-06
1.01444504708e-06
1.01414377696e-06
1.01383193396e-06
1.01350951809e-06
1.01317675671e-06
1.01283330878e-06
1.01247940165e-06
1.01211469428e-06
1.01173895928e-06
1.01135219666e-06
1.01095406535e-06
1.01054433799e-06
1.01012290088e-06
1.00968952665e-06
1.00924387425e-06
1.00878605735e-06
1.00831550753e-06
1.00783222479e-06
1.00733620911e-06
1.00682689208e-06
1.00630450106e-06
1.00576858131e-06
1.00521913282e-06
1.00465547348e-06
1.00407783066e-06
1.00348563592e-06
1.00287866189e-06
1.0022566812e-06
1.00161901173e-06
1.00096565347e-06
1.000296038e-06
9.99609596875e-07
9.98905875349e-07
9.9818430499e-07
9.97444658424e-07
9.96685912469e-07
9.95907726065e-07
9.95109530777e-07
9.94290417111e-07


In [30]:
movieratings = pd.read_csv('ratings.csv')

In [31]:
def getDfSummary(input_data):
    output_data = input_data.describe(include = 'all').T
    var = pd.DataFrame(data = {'nanvals': pd.Series(), 'number_distinct': pd.Series()})
    for i in range(len(input_data.columns)):
        nanvals = input_data.ix[:,i].isnull().sum()
        number_distinct = len(input_data.ix[:,i].value_counts())
        var = var.append(pd.DataFrame([[nanvals, number_distinct]], columns = ['nanvals', 'number_distinct']))
    var.index = output_data.index.values
    output_data['nanvals'] = var['nanvals']
    output_data['number_distinct'] = var['number_distinct']
    return output_data
output_data = getDfSummary(movieratings)

In [32]:
users = movieratings.ix[:,0].values
songs = movieratings.ix[:,1].values
ratings = movieratings.ix[:,2].values

#unique users / songs
uni_users = movieratings['userId'].unique()
uni_songs = movieratings['movieId'].unique()

#dict mapping the id to an index
user_map = dict(zip(uni_users,range(len(uni_users))))
song_map = dict(zip(uni_songs,range(len(uni_songs))))

user_idx =  np.array([ user_map[user] for user in users])
song_idx =  np.array([ song_map[song] for song in songs])

print(len(uni_users),len(uni_songs))

perm = np.random.permutation(len(users))
trn_idx = perm[:(len(users)*2)//3]
val_idx = perm[(len(users)*2)//3:]
user_idx_trn, song_idx_trn, ratings_trn = user_idx[trn_idx], song_idx[trn_idx], ratings[trn_idx]
user_idx_val, song_idx_val, ratings_val = user_idx[val_idx], song_idx[val_idx], ratings[val_idx]

671 9066


In [33]:
songmodel = MF_RS(len (uni_users), len(uni_songs), 11, reg_lambda=0.001, conf_lambda=1000)

In [34]:
songmodel.evaluate(user_idx_val, song_idx_val, ratings_val)

31.485473169923488

In [35]:
np.random.seed(1)
songmodel.train(user_idx, song_idx, ratings)

0.0177569353469
0.00728894162644
0.00437736500334
0.00299338117335
0.00219362973096
0.00167895445717
0.0013241497355
0.00106871611741
0.000879055602825
0.000735522116069
0.000625660242455
0.000540281646012
0.000473986714263
0.000421110533236
0.000379479870317
0.000346367369522
0.000319600734569
0.000298156250181
0.000281122526503
0.00026709278021
0.000254938855913
0.00024617767951
0.000238400287955
0.000231304382702
0.000226440661354
0.000221555254335
0.000217854571005
0.000214787934179
0.000212105687271
0.000209964586335
0.000207458505785
0.000206390633044
0.000204533244047
0.000203461896337
0.000202069821913
0.000201524780074
0.000200898054696
0.000199722880381
0.000198729251046
0.000198358584224
0.000197979695542
0.000198002902791
0.000197466544545
0.000196422465116
0.000196156546823
0.000196425579459
0.000196051272447
0.000195372224232
0.000195045375614
0.000194953288534


In [36]:
songmodel.evaluate(user_idx_val, song_idx_val, ratings_val)

0.2804775435135487